In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get rid of franchised stores

In [3]:
raw_df = pd.read_csv("4_23_cleaned_data.csv")

franchised_stores_df = pd.read_excel("franchised_stores.xlsx")

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/4162898329.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("4_23_cleaned_data.csv")


In [4]:
df_no_franchise = raw_df[~raw_df['Location ID'].isin(franchised_stores_df['Store'])]

## Turn dates into datetime, filter for dates between Feb 2021 and Aug 2023

In [5]:
type(df_no_franchise.loc[0, 'Completed Date'])

str

In [6]:
df_no_franchise['Completed Date'] = pd.to_datetime(df_no_franchise['Completed Date'])

# Define the date range
start_date = '2021-02-01'
end_date = '2023-09-01'

# Filter the DataFrame for the date range
filtered_df = df_no_franchise[(df_no_franchise['Completed Date'] >= start_date) & (df_no_franchise['Completed Date'] <= end_date)]

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/3085906510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_franchise['Completed Date'] = pd.to_datetime(df_no_franchise['Completed Date'])


## Drop Irrelevant Columns

In [7]:
df = filtered_df[["Location ID", "Tonnage", "Completed Date", "Zip code", "State", "Latitude", "Longitude"]]

In [8]:
df

,Location ID,Tonnage,Completed Date,Zip code,State,Latitude,Longitude
9110,5051,3.51,2021-02-01,80634.0,CO,40.422824,-104.754675
9111,1370,7.48,2021-02-01,86442.0,AZ,35.082515,-114.594161
9113,3620,8.78,2021-02-01,84065.0,UT,40.505236,-111.977516
9114,6090,10.60,2021-02-01,84307.0,UT,41.560951,-112.143230
9115,2767,8.49,2021-02-01,85215.0,AZ,33.449221,-111.733746
...,...,...,...,...,...,...,...
364994,3478,1.02,2023-09-01,NaN,HI,21.294903,-157.843233
364995,5274,3.03,2023-09-01,NaN,HI,21.397174,-157.974452
364996,2473,5.01,2023-09-01,NaN,HI,19.700194,-155.061954
364997,2071,5.66,2023-09-01,NaN,AK,61.140195,-149.869863


## Do the stuff

In [9]:
# Sort the DataFrame
df.sort_values(by=['Location ID', 'Completed Date'], inplace=True)

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/1444988040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['Location ID', 'Completed Date'], inplace=True)


In [10]:
# Calculate the number of days between current and last pickup
df['Days Since Last Pickup'] = df.groupby('Location ID')['Completed Date'].diff().dt.days

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/1293491196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Since Last Pickup'] = df.groupby('Location ID')['Completed Date'].diff().dt.days


In [11]:
# Extract information about the last three pickups
df['Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(1)
df['Second Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(2)
df['Third Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(3)

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/1317808763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(1)
/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/1317808763.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Second Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(2)
/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/1317808763.py:4: SettingWithCopyWarn

In [12]:
weekmask = '1111100'  # Monday=1, Tuesday=1, Wednesday=1, Thursday=1, Friday=1, Saturday=0, Sunday=0

def calculate_weekdays(row):
    if pd.isna(row['Days Since Last Pickup']):
        return np.nan
    start_date = row['Completed Date'] - pd.to_timedelta(row['Days Since Last Pickup'], unit='d')
    end_date = row['Completed Date']
    return np.busday_count(start_date.date(), end_date.date(), weekmask=weekmask)

df['Weekdays Since Last Pickup'] = df.apply(calculate_weekdays, axis=1)

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/3490519654.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weekdays Since Last Pickup'] = df.apply(calculate_weekdays, axis=1)


In [13]:
df['Days Before Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(1)
df['Days Before Second Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(2)
df['Days Before Third Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(3)

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/3768080695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Before Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(1)
/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/3768080695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Before Second Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(2)
/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_1196

In [14]:
df['Month'] = df['Completed Date'].dt.month

/var/folders/zm/th24w6tj1nj2x7f_3wfyx1mh0000gn/T/ipykernel_11968/3978473839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Completed Date'].dt.month


In [15]:
df

,Location ID,Tonnage,Completed Date,Zip code,State,Latitude,Longitude,Days Since Last Pickup,Last Pickup Tonnage,Second Last Pickup Tonnage,Third Last Pickup Tonnage,Weekdays Since Last Pickup,Days Before Last Pickup,Days Before Second Last Pickup,Days Before Third Last Pickup,Month
11032,1,8.90,2021-02-05,NaN,AR,36.331300,-94.149054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11936,1,9.15,2021-02-08,72756.0,AR,36.331300,-94.149054,3.0,8.90,NaN,NaN,1.0,NaN,NaN,NaN,2
14950,1,5.32,2021-02-19,72756.0,AR,36.331300,-94.149054,11.0,9.15,8.90,NaN,9.0,3.0,NaN,NaN,2
17549,1,5.61,2021-02-26,72756.0,AR,36.331300,-94.149054,7.0,5.32,9.15,8.90,5.0,11.0,3.0,NaN,2
20612,1,6.69,2021-03-08,72756.0,AR,36.331300,-94.149054,10.0,5.61,5.32,9.15,6.0,7.0,11.0,3.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319352,30095,13.38,2023-05-26,72712.0,AR,36.346243,-94.195376,120.0,12.62,11.31,3.99,86.0,115.0,224.0,173.0,5
325218,30095,1.01,2023-06-09,72712.0,AR,36.346243,-94.195376,14.0,13.38,12.62,11.31,10.0,120.0,115.0,224.0,6
181439,30099,10.79,2022-06-06,72712.0,AR,36.340990,-94.200230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
247041,30099,7.46,2022-11-25,72712.0,AR,36.340990,-94.200230,172.0,10.79,NaN,NaN,124.0,NaN,NaN,NaN,11


In [16]:
df[df["Location ID"] == 1]

,Location ID,Tonnage,Completed Date,Zip code,State,Latitude,Longitude,Days Since Last Pickup,Last Pickup Tonnage,Second Last Pickup Tonnage,Third Last Pickup Tonnage,Weekdays Since Last Pickup,Days Before Last Pickup,Days Before Second Last Pickup,Days Before Third Last Pickup,Month
11032,1,8.90,2021-02-05,NaN,AR,36.3313,-94.149054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11936,1,9.15,2021-02-08,72756.0,AR,36.3313,-94.149054,3.0,8.90,NaN,NaN,1.0,NaN,NaN,NaN,2
14950,1,5.32,2021-02-19,72756.0,AR,36.3313,-94.149054,11.0,9.15,8.90,NaN,9.0,3.0,NaN,NaN,2
17549,1,5.61,2021-02-26,72756.0,AR,36.3313,-94.149054,7.0,5.32,9.15,8.90,5.0,11.0,3.0,NaN,2
20612,1,6.69,2021-03-08,72756.0,AR,36.3313,-94.149054,10.0,5.61,5.32,9.15,6.0,7.0,11.0,3.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348949,1,6.46,2023-08-03,72756.0,AR,36.3313,-94.149054,7.0,5.61,6.49,6.58,5.0,7.0,7.0,7.0,8
351751,1,5.36,2023-08-10,72756.0,AR,36.3313,-94.149054,7.0,6.46,5.61,6.49,5.0,7.0,7.0,7.0,8
355136,1,6.51,2023-08-17,72756.0,AR,36.3313,-94.149054,7.0,5.36,6.46,5.61,5.0,7.0,7.0,7.0,8
357896,1,5.30,2023-08-23,72756.0,AR,36.3313,-94.149054,6.0,6.51,5.36,6.46,4.0,7.0,7.0,7.0,8


In [17]:
df.to_csv("processed_data.csv")